In [29]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext jupyternotify
from task2_utils_niklas import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression,Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from scipy.special import expit as sigmoid

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [2]:
train_features = pd.read_csv("./handout/train_features.csv",index_col='pid')
trains_labels = pd.read_csv("./handout/train_labels.csv",index_col='pid')
test_features = pd.read_csv("./handout/test_features.csv",index_col='pid')

In [3]:
#print("number of unique pid is {}".format(np.size(train_features.pid.unique())))
print("train_features has {} rows and {} columns".format(train_features.shape[0],train_features.shape[1]))
train_features.describe()

train_features has 227940 rows and 36 columns


Time            Age        EtCO2           PTT           BUN       Lactate          Temp           Hgb         HCO3    BaseExcess          RRate   Fibrinogen     Phosphate           WBC    Creatinine         PaCO2          AST          FiO2     Platelets          SaO2       Glucose           ABPm     Magnesium     Potassium           ABPd       Calcium  Alkalinephos           SpO2  Bilirubin_direct      Chloride           Hct      Heartrate  Bilirubin_total    TroponinI           ABPs            pH
count  227940.000000  227940.000000  9783.000000  10299.000000  20105.000000  10756.000000  81115.000000  22295.000000  12559.00000  19887.000000  187785.000000  2493.000000  11590.000000  19083.000000  17792.000000  21043.000000  5761.000000  26602.000000  18035.000000  13014.000000  47036.000000  195889.000000  17523.000000  28393.000000  152418.000000  17830.000000   5708.000000  195192.000000        719.000000  13917.000000  27297.000000  200128.000000      5326.000000  3776.000000  191650.000000  25046.000000
mean        7.014399      62.073809    32.883114     40.091310     23.192664      2.859716     36.852136     10.628208     23.48810     -1.239284      18.154043   262.496911      3.612519     11.738649      1.495777     41.115696   193.444888      0.701666    204.666426     93.010527    142.169407      82.117276      2.004149      4.152729      64.014711      7.161149     97.796163      97.663449          1.390723    106.260185     31.283090      84.522371         1.640941     7.269240     122.369877      7.367231
std         4.716103      16.451854     7.802065     26.034961     20.024289      2.428368      0.875152      2.074859      4.40378      4.192677       5.037031   133.020910      1.384462     10.088872      1.898112      8.929873   682.836708     24.522126    104.156406     10.887271     56.894530      16.471871      0.437286      0.670168      13.920097      2.812067    122.773379       2.786186          2.792722      5.916082      5.770425      17.643437         3.244145    25.172442      23.273834      0.074384
min         1.000000      15.000000    10.000000     12.500000      1.000000      0.200000     21.000000      3.300000      0.00000    -29.000000       1.000000    34.000000      0.200000      0.100000      0.100000     10.000000     5.000000      0.000000      2.000000     24.000000     15.000000      20.000000      0.500000      1.300000      20.000000      1.000000     12.000000      20.000000          0.010000     66.000000      9.400000      23.000000         0.100000     0.010000      21.000000      6.820000
25%         4.000000      52.000000    28.500000     27.800000     12.000000      1.400000     36.000000      9.200000     21.00000     -3.000000      15.000000   177.000000      2.800000      7.600000      0.700000     36.000000    21.000000      0.400000    137.000000     95.000000    108.000000      71.000000      1.700000      3.700000      54.250000      7.300000     53.000000      96.000000          0.100000    103.000000     27.300000      72.000000         0.500000     0.030000     105.000000      7.330000
50%         7.000000      64.000000    33.000000     32.200000     17.000000      2.100000     37.000000     10.500000     23.90000     -1.000000      18.000000   233.000000      3.400000     10.400000      0.900000     40.000000    36.000000      0.500000    189.000000     97.000000    130.000000      80.000000      2.000000      4.100000      62.000000      8.200000     72.000000      98.000000          0.300000    107.000000     30.900000      83.000000         0.800000     0.150000     119.000000      7.370000
75%        10.000000      74.000000    38.000000     40.600000     27.000000      3.400000     37.000000     12.000000     26.00000      0.000000      21.000000   316.000000      4.200000     14.000000      1.380000     45.000000    84.000000      0.600000    251.000000     98.000000    160.000000      91.000000      2.200000      4.500000      72.000000      8.700000    10

In [4]:
train_features.isna().sum().to_frame().transpose()

Time  Age   EtCO2     PTT     BUN  Lactate    Temp     Hgb    HCO3  BaseExcess  RRate  Fibrinogen  Phosphate     WBC  Creatinine   PaCO2     AST    FiO2  Platelets    SaO2  Glucose   ABPm  Magnesium  Potassium   ABPd  Calcium  Alkalinephos   SpO2  Bilirubin_direct  Chloride     Hct  Heartrate  Bilirubin_total  TroponinI   ABPs      pH
0     0    0  218157  217641  207835   217184  146825  205645  215381      208053  40155      225447     216350  208857      210148  206897  222179  201338     209905  214926   180904  32051     210417     199547  75522   210110        222232  32748            227221    214023  200643      27812           222614     224164  36290  202894

In [5]:
# print("number of unique pid is {}".format(np.size(trains_labels.pid.unique())))
print("trains_labels has {} rows and {} columns".format(trains_labels.shape[0],trains_labels.shape[1]))
trains_labels.describe()

trains_labels has 18995 rows and 15 columns


LABEL_BaseExcess  LABEL_Fibrinogen     LABEL_AST  LABEL_Alkalinephos  LABEL_Bilirubin_total  LABEL_Lactate  LABEL_TroponinI    LABEL_SaO2  LABEL_Bilirubin_direct   LABEL_EtCO2  LABEL_Sepsis   LABEL_RRate    LABEL_ABPm    LABEL_SpO2  LABEL_Heartrate
count      18995.000000      18995.000000  18995.000000         18995.00000           18995.000000   18995.000000     18995.000000  18995.000000            18995.000000  18995.000000  18995.000000  18995.000000  18995.000000  18995.000000     18995.000000
mean           0.268281          0.073704      0.239747             0.23622               0.240590       0.200211         0.099763      0.233693                0.033904      0.066017      0.057278     18.795960     82.511171     96.947311        84.119716
std            0.443076          0.261295      0.426940             0.42477               0.427453       0.400168         0.299692      0.423190                0.180986      0.248319      0.232380      3.511241     12.745110      2.110957        14.718396
min            0.000000          0.000000      0.000000             0.00000               0.000000       0.000000         0.000000      0.000000                0.000000      0.000000      0.000000      1.000000     26.000000     27.000000        30.200000
25%            0.000000          0.000000      0.000000             0.00000               0.000000       0.000000         0.000000      0.000000                0.000000      0.000000      0.000000     16.550000     73.200000     95.900000        73.700000
50%            0.000000          0.000000      0.000000             0.00000               0.000000       0.000000         0.000000      0.000000                0.000000      0.000000      0.000000     18.400000     81.000000     97.100000        83.400000
75%            1.000000          0.000000      0.000000             0.00000               0.000000       0.000000         0.000000      0.000000                0.000000      0.000000      0.000000     20.600000     90.200000     98.300000        93.600000
max            1.000000          1.000000      1.000000             1.00000               1.000000       1.000000         1.000000      1.000000                1.000000      1.000000      1.000000     41.100000    147.100000    100.000000       155.600000

In [6]:
# print("number of unique pid is {}".format(np.size(test_features.pid.unique())))
print("test_features has {} rows and {} columns".format(test_features.shape[0],test_features.shape[1]))
test_features.describe()

test_features has 151968 rows and 36 columns


Time            Age        EtCO2          PTT           BUN      Lactate          Temp           Hgb         HCO3    BaseExcess          RRate   Fibrinogen    Phosphate           WBC    Creatinine         PaCO2          AST          FiO2     Platelets         SaO2       Glucose           ABPm     Magnesium     Potassium           ABPd       Calcium  Alkalinephos           SpO2  Bilirubin_direct     Chloride           Hct      Heartrate  Bilirubin_total    TroponinI           ABPs            pH
count  151968.000000  151968.000000  6528.000000  6853.000000  13448.000000  7276.000000  54155.000000  14872.000000  8422.000000  13238.000000  125299.000000  1655.000000  7686.000000  12753.000000  11805.000000  13881.000000  3908.000000  17799.000000  11873.000000  8670.000000  31072.000000  130672.000000  11827.000000  19015.000000  101309.000000  11892.000000   3861.000000  130227.000000         478.00000  9417.000000  18249.000000  133565.000000      3549.000000  2469.000000  128317.000000  16806.000000
mean        7.000553      62.154296    32.560432    39.934071     23.438467     2.743194     36.853698     10.655362    23.494028     -1.129381      18.201614   263.741571     3.612015     11.638304      1.478575     41.122434   251.066658      0.553304    207.103660    92.981430    142.704627      81.858080      2.012246      4.153665      63.708425      7.158564     99.091557      97.668464           1.46046   106.321705     31.373145      84.525235         1.670710     7.410737     122.179033      7.369232
std         4.436079      16.338174     7.656557    25.250865     19.992525     2.288238      0.875930      2.060924     4.268973      4.074558       5.099404   135.765163     1.363589      6.597006      1.833298      8.936093   912.050978      0.215522    105.003544    10.722283     58.320583      16.347929      0.436171      0.677888      13.790649      2.816791    111.488116       2.781903           2.83072     5.913163      5.759315      17.570528         3.297235    24.346532      23.057958      0.072396
min         1.000000      16.000000    10.000000    18.100000      1.000000     0.300000     27.000000      2.300000     0.000000    -32.000000       1.000000    35.000000     0.300000      0.100000      0.100000     14.000000     5.000000      0.000000      5.000000    27.000000     14.000000      20.000000      0.200000      1.400000      20.000000      1.000000     11.000000      22.000000           0.01000    74.000000      9.100000      21.000000         0.100000     0.010000      28.000000      6.620000
25%         4.000000      52.000000    28.000000    27.600000     12.000000     1.310000     36.000000      9.200000    21.000000     -3.000000      15.000000   174.000000     2.800000      7.700000      0.700000     36.000000    21.000000      0.400000    138.000000    95.000000    108.000000      70.000000      1.700000      3.700000      54.000000      7.300000     53.000000      96.000000           0.12000   103.000000     27.400000      72.000000         0.500000     0.030000     105.000000      7.330000
50%         7.000000      64.000000    33.000000    32.000000     17.000000     2.020000     37.000000     10.500000    24.000000     -1.000000      18.000000   228.000000     3.400000     10.500000      0.920000     40.000000    36.000000      0.500000    193.000000    97.000000    130.500000      80.000000      2.000000      4.100000      62.000000      8.200000     72.000000      98.000000           0.30000   107.000000     31.000000      83.000000         0.800000     0.210000     119.000000      7.370000
75%        10.000000      74.000000    37.000000    40.600000     27.000000     3.300000     37.000000     12.000000    26.000000      0.500000      21.000000   313.000000     4.200000     14.200000      1.380000     45.000000    88.000000      0.600000    256.000000    98.000000    160.000000      91.000000      2.200000      4.500000      71.500000      8.700000    103.000000     100.000000           1.300

In [7]:
features_train = train_features.copy(deep=True)
features_test = test_features.copy(deep=True)

features_train = features_train.apply(lambda col: sigmoid((col - np.mean(col))/np.std(col)))
features_test = features_test.apply(lambda col: sigmoid((col - np.mean(col))/np.std(col)))

In [11]:
%notify
start = time.time()
from fancyimpute import IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0,sample_posterior=False)

# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=2)

# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')


features_train.iloc[:, :] = imputer.fit_transform(features_train)
# features_test.iloc[:, :] = imputer.fit_transform(imputed_test)
features_test.iloc[:, :] = imputer.transform(features_test)

end = time.time()
print('It took the iterative imputer '+str(end-start)+' seconds to impute the data.')

<IPython.core.display.Javascript object>

It took the iterative imputer 51.05798578262329 seconds to impute the data.


In [12]:
print(train_features.shape," ",np.size(train_features.index.unique()))
print(test_features.shape," ",np.size(test_features.index.unique()))

(227940, 36)   18995
(151968, 36)   12664


In [13]:
# scaler = MinMaxScaler()

# features_train_scaled = scaler.fit_transform(features_train)
# features_train_scaled = pd.DataFrame(features_train_scaled,features_train.index,columns=features_train.columns)

# features_test_scaled = scaler.transform(features_test)
# features_test_scaled = pd.DataFrame(features_test_scaled,features_test.index,columns=features_test.columns)

In [14]:
%notify
start = time.time()

fullTrain_concat = concat_pid_rows(features_train)
fullTest_concat = concat_pid_rows(features_test)

## for skipping PCA, uncomment this:
fullTrain = fullTrain_concat
fullTest = fullTest_concat

end = time.time()
print('It took the iterative imputer '+str(end-start)+' seconds to concat the data.')

<IPython.core.display.Javascript object>

It took the iterative imputer 19.319281816482544 seconds to concat the data.


In [15]:
%notify

pca = PCA(n_components = 0.99)

fullTrain_pca = pca.fit_transform(fullTrain_concat)
fullTrain = pd.DataFrame(fullTrain_pca,fullTrain_concat.index)

fullTest_pca = pca.transform(fullTest_concat)
fullTest = pd.DataFrame(fullTest_pca,fullTest_concat.index)

<IPython.core.display.Javascript object>

In [16]:
print(features_train.shape," ",fullTrain_concat.shape," ",fullTrain_pca.shape," ",fullTrain.shape," ")
print(features_test.shape," ",fullTest_concat.shape," ",fullTest_pca.shape," ",fullTest.shape," ")

(227940, 36)   (18995, 409)   (18995, 263)   (18995, 263)  
(151968, 36)   (12664, 409)   (12664, 263)   (12664, 263)  


In [17]:
fullTrain.to_pickle('./pickled_data/fullTrain_Farzam')
fullTest.to_pickle('./pickled_data/fullTest_Farzam')

# fullTrain = pd.read_pickle('./pickled_data/fullTrain_Farzam')
# fullTest = pd.read_pickle('./pickled_data/fullTest_Farzam')

# fullTrain = pd.read_pickle('./pickled_data/features_train_scaled_impiter_concat_pca.pkl')
# fullTest = pd.read_pickle('./pickled_data/features_test_scaled_impiter_concat_pca.pkl')

# fullTrain = pd.read_pickle('./pickled_data/features_train_scaled_impknn_concat_pca')
# fullTest = pd.read_pickle('./pickled_data/features_test_scaled_impknn_concat_pca')

In [19]:
# runs faster
%notify

results_rbf = pd.DataFrame(columns=trains_labels.columns)
n_estimators = 10

for i in np.arange(0,11):
    
        print(trains_labels.columns[i])
        X = fullTrain.iloc[:,:]
        y = trains_labels.iloc[:,i]
        X_test_ = fullTest
        
        X, y = SMOTE().fit_resample(X, y)
        # X, y = ADASYN().fit_resample(X, y)
        # X, y = BorderlineSMOTE().fit_resample(X, y)
        # smote_nc = SMOTENC(categorical_features=[0, 2], random_state=0)
        # X, y = = smote_nc.fit_resample(X, y)
        
        clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='rbf',C=1e5,degree=5,gamma='scale',shrinking=True,
                                    probability=True,class_weight='balanced'), 
                                    max_samples=1.0 / n_estimators, n_estimators=n_estimators,n_jobs=-1,bootstrap = False))
        clf.fit(X, y)
        results_rbf[trains_labels.columns[i]] = sigmoid(clf.decision_function(X_test_))

<IPython.core.display.Javascript object>

LABEL_BaseExcess
LABEL_Fibrinogen
LABEL_AST
LABEL_Alkalinephos
LABEL_Bilirubin_total
LABEL_Lactate
LABEL_TroponinI
LABEL_SaO2
LABEL_Bilirubin_direct
LABEL_EtCO2
LABEL_Sepsis


# Appendix

## Small scale tests

In [ ]:
X = fullTrain.iloc[:,:]
y = trains_labels.iloc[:,0]

X, y = SMOTE().fit_resample(X, y)
# X, y = ADASYN().fit_resample(X, y)
# X, y = BorderlineSMOTE().fit_resample(X, y)
# smote_nc = SMOTENC(categorical_features=[0, 1], random_state=0)
# X, y = smote_nc.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
n_estimators = 10
clf_linear = OneVsRestClassifier(BaggingClassifier(SVC(kernel='rbf',C=1e5,degree=5,gamma='scale',shrinking=True,
                                    probability=True,class_weight='balanced'), 
                                    max_samples=1.0 / n_estimators, n_estimators=n_estimators,n_jobs=-1,bootstrap = False))
clf_linear.fit(X_train, y_train)

In [ ]:
print("train ROC score: ",roc_auc_score(y_train, pd.DataFrame(clf_linear.predict_proba(X_train)).iloc[:,1]))
print("test ROC score: ",roc_auc_score(y_test, pd.DataFrame(clf_linear.predict_proba(X_test)).iloc[:,1]))
print("test ROC score (with decision function): ",roc_auc_score(y_test, sigmoid(clf_linear.decision_function(X_test))))

## Logistic Regression

In [ ]:
model = LogisticRegression(solver='lbfgs',penalty="l2",max_iter=10000,C=1e10,n_jobs=-1)
model.fit(X_train, y_train)

print("train ROC score: ",roc_auc_score(y_train, pd.DataFrame(model.predict_proba(X_train)).iloc[:,1]))
print("test ROC score: ",roc_auc_score(y_test, pd.DataFrame(model.predict_proba(X_test)[:, 1])))

#  subtask 2

In [27]:
X = fullTrain.iloc[:,:]
y = trains_labels.loc[:,'LABEL_Sepsis']
X_test_ = fullTest
param_grid_C_class = {'C':[2**-1, 2**1, 2**3, 2**5, 2**7, 2**9, 2**11]}

clf_sepsis = GridSearchCV(LogisticRegression(class_weight='balanced'),param_grid_C_class,cv=3)
clf_sepsis.fit(X,y)
results_rbf['LABEL_Sepsis'] = sigmoid(clf_sepsis.decision_function(X_test_))
print("train ROC score: ",roc_auc_score(y, pd.DataFrame(clf_sepsis.predict_proba(X)).iloc[:,1]))

/Users/farzamf/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/farzamf/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

/Users/farzamf/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/farzamf/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

train ROC score:  0.6476868641576249


/Users/farzamf/Anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#  subtask 3

In [30]:
labels_task3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
param_grid_alpha_reg = {'alpha':[ 2**1, 2**3, 2**5, 2**7, 2**9, 2**11, 2**13]} 
X = fullTrain.iloc[:,:]
X_test_ = fullTest

for label in labels_task3:

    print('>>> Starting to fit test {}'.format(label))
    clf = GridSearchCV(Ridge(),param_grid_alpha_reg,)
    clf.fit(X,trains_labels.loc[:,label])
    print(' > Best estimator: {}'.format(clf.best_estimator_))
    results_rbf[label] = clf.predict(X_test_)

>>> Starting to fit test LABEL_RRate
 > Best estimator: Ridge(alpha=8192, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
>>> Starting to fit test LABEL_ABPm
 > Best estimator: Ridge(alpha=8192, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
>>> Starting to fit test LABEL_SpO2
 > Best estimator: Ridge(alpha=8192, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
>>> Starting to fit test LABEL_Heartrate
 > Best estimator: Ridge(alpha=2048, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)


In [39]:
results = results_rbf
results.fillna(0,inplace=True)
results.to_csv('./toSubmit/prediction.zip', index=False, float_format='%.3f', compression='zip')
results

LABEL_BaseExcess  LABEL_Fibrinogen  LABEL_AST  LABEL_Alkalinephos  LABEL_Bilirubin_total  LABEL_Lactate  LABEL_TroponinI  LABEL_SaO2  LABEL_Bilirubin_direct  LABEL_EtCO2  LABEL_Sepsis  LABEL_RRate  LABEL_ABPm  LABEL_SpO2  LABEL_Heartrate    pid
0              0.150890          0.100253   0.131936            0.134838               0.133186       0.128548         0.089487    0.130740                0.122139     0.100692      0.005172    18.784869   82.513843   96.954829        84.113211      0
1              0.390834          0.267694   0.412138            0.424509               0.439374       0.383112         0.321595    0.421316                0.176526     0.133046      0.447713    18.844643   82.414979   96.939671        83.885538      3
2              0.571094          0.691411   0.541095            0.593504               0.575790       0.555271         0.478076    0.568370                0.429210     0.483810      0.405907    18.808944   82.682995   96.926375        84.353815      5
3              0.149940          0.099956   0.131729            0.134521               0.133428       0.128297         0.089525    0.129958                0.121849     0.100153      0.724362    18.712187   82.447681   97.016613        82.913048      7
4              0.453103          0.539020   0.479904            0.453804               0.452856       0.550867         0.428957    0.457851                0.503153     0.401615      0.452530    18.782429   82.497853   96.942627        83.787898      9
...                 ...               ...        ...                 ...                    ...            ...              ...         ...                     ...          ...           ...          ...         ...         ...              ...    ...
12659          0.633850          0.582222   0.504933            0.556851               0.554026       0.512937         0.526744    0.502483                0.472234     0.496658      0.505934    18.809872   82.573514   96.938903        84.259937  31647
12660          0.322282          0.205397   0.323879            0.416708               0.341733       0.238484         0.292823    0.215430                0.244617     0.409813      0.447150    18.817634   82.554978   96.950484        84.083862  31649
12661          0.149170          0.099806   0.131452            0.134322               0.133073       0.127888         0.089390    0.129362                0.121609     0.099909      0.989923    18.762565   82.307475   96.919595        83.283877  31651
12662          0.390519          0.325650   0.452005            0.422097               0.445964       0.302585         0.396698    0.382083                0.381118     0.339173      0.574964    18.730143   82.491266   96.982300        83.598555  31652
12663          0.565897          0.415378   0.536355            0.502073               0.579859       0.581216         0.392831    0.480455                0.350683     0.479565      0.356384    18.764060   82.320245   96.987785        83.739803  31655

[12664 rows x 16 columns]

In [32]:
# %load handout/score_submission.py
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']


def get_score(df_true, df_submission):
    df_submission = df_submission.sort_values('pid')
    df_true = df_true.sort_values('pid')
    task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
    task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
    task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
    score = np.mean([task1, task2, task3])
    print(task1, task2, task3)
    return score


filename = 'handout/sample.zip'
# df_submission = pd.read_csv("results.csv")
df_submission = results

# generate a baseline based on sample.zip
df_true = pd.read_csv(filename)
for label in TESTS + ['LABEL_Sepsis']:
    # round classification labels
    df_true[label] = np.around(df_true[label].values)

print('Score of sample.zip with itself as groundtruth', get_score(df_true, df_submission))

0.501719625682931 0.4988797355062414 0.5
Score of sample.zip with itself as groundtruth 0.5001997870630575
